### Load libraries

In [96]:
#load libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import os
import seaborn as sns
import geopandas as gpd

### Read data

In [97]:
#read the ICER data
icer = pd.read_excel("../data/raw/ICER_data.xlsx")

In [98]:
#read in the data for life expectancy
le_district = pd.read_csv("../data/processed/le_combined.csv")
le_nat = pd.read_csv("../data/processed/le_calc_national.csv")
df_map = gpd.read_file("../data/processed/high_res.shp")

In [ ]:
#normalise population density first


In [99]:
#rename first column as treatment
icer = icer.rename(columns={"Unnamed: 0": "Treatment"})

In [100]:
icer.head()

,Treatment,burden,qaly_lost,healthcare_cost,payer_cost
0,non_vaccine,382416,184088.54,26618814,16066984
1,vaccine,264217,127189.68,352828922,345497041


### Calculate district-level ICER based on the available ICER (average) from 2020-2024 in Indonesia (from research paper), nationwide, adjusted by life expectancy to obtain district-level ICER, assuming intervention costs remain approximately the same (similar) --> life expectancy is to adjust based on QALYs. All prices are adjusted to 2018 USD based on PPP and discounted by 3%

In [101]:
le_district.head()

,District,Year,Life Expectancy
0,ACEH,2022,70.18
1,SIMEULUE,2022,65.48
2,ACEH SINGKIL,2022,67.65
3,ACEH SELATAN,2022,64.64
4,ACEH TENGGARA,2022,68.48


In [102]:
#rename as district_life_exp
le_district = le_district.rename(columns={"Life Expectancy": "district_le"})

In [103]:
le_nat.head()

,Year,Life Expectancy
0,2019,69.213148
1,2020,69.363809
2,2021,69.474243
3,2022,69.747513
4,2023,70.018191


In [104]:
#rename as nat_le
le_nat = le_nat.rename(columns={"Life Expectancy": "nat_le"})

In [105]:
#merge le_district and le_nat by year
le_final = pd.merge(le_district, le_nat, on = "Year")

#replace all the NA values

#make sure all the life expectancy values are all floats
le_final["district_le"] = le_final["district_le"].astype(float)
le_final["nat_le"] = le_final["nat_le"].astype(float)

#calculate the le_factor, which is a comparison of the district-level to the national-level life expectancy
le_final["le_factor"] = le_final["district_le"] / le_final["nat_le"]

In [106]:
#create another column in df_map to calculate the discounting factor for each district in dki jakarta and west java
df = pd.merge(df_map, le_final, on = ("District", "Year"))

In [107]:
#remove the district_le and nat_le columns
df_final = df.drop(columns = ["district_le", "nat_le"])

In [113]:
#get per capita, yearly cost from nationwide, aggregated data (2020-2024)
df_final["burden_non_vaccine"] = icer["burden"].iloc[0]
df_final["burden_vaccine"] = icer["burden"].iloc[1]
df_final["healthcare_cost_non_vaccine"] = icer["healthcare_cost"].iloc[0] / df_final["burden_non_vaccine"] / 5
df_final["healthcare_cost_vaccine"] = icer["healthcare_cost"].iloc[1] / df_final["burden_vaccine"] / 5
df_final["payer_cost_non_vaccine"] = icer["payer_cost"].iloc[0] / df_final["burden_non_vaccine"] / 5
df_final["payer_cost_vaccine"] = icer["payer_cost"].iloc[1] / df_final["burden_vaccine"] / 5


In [ ]:
# Incremental cost per dengue case (per capita)  yearly
df_final["incremental_healthcare_cost"] = (
    df_final["healthcare_cost_vaccine"] - df_final["healthcare_cost_non_vaccine"]) * df_final["Cases"]

df_final["incremental_payer_cost"] = (
    df_final["payer_cost_vaccine"] - df_final["payer_cost_non_vaccine"]) * df_final["Cases"]

In [115]:
#scale this using population density, urbanicity, and differences in dengue case counts
df_final.head()

,District,Year,Area_sq_km,HDI,Province,Cases,Population,Pop_den,urbanicity,geometry,le_factor,burden_non_vaccine,burden_vaccine,healthcare_cost_non_vaccine,healthcare_cost_vaccine,payer_cost_non_vaccine,payer_cost_vaccine,incremental_healthcare_cost,incremental_payer_cost
0,BANDUNG,2020,1767.96,0.7239,Jawa Barat,2295,3623790,2.049701,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",1.060063,382416,264217,13.921391,267.07511,8.402883,261.525217,5.809878e+05,5.809158e+05
1,BANDUNG,2021,1767.96,0.7273,Jawa Barat,2002,3665655,2.073381,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",1.061113,382416,264217,13.921391,267.07511,8.402883,261.525217,5.068137e+05,5.067509e+05
2,BANDUNG,2022,1767.96,0.7316,Jawa Barat,4191,3707523,2.097063,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",1.061113,382416,264217,13.921391,267.07511,8.402883,261.525217,1.060967e+06,1.060836e+06
3,BANDUNG,2023,1767.96,0.7374,Jawa Barat,1005,3749391,2.120744,0.123155,"POLYGON ((107.73309 -6.814, 107.73354 -6.81427...",1.060724,382416,264217,13.921391,267.07511,8.402883,261.525217,2.544195e+05,2.543879e+05
4,BANDUNG BARAT,2020,1305.77,0.6808,Jawa Barat,966,1788336,1.369564,0.079001,"POLYGON ((107.40945 -6.68851, 107.40986 -6.688...",1.042907,382416,264217,13.921391,267.07511,8.402883,261.525217,2.445465e+05,2.445162e+05


### Save the df for other icer calculations

In [ ]:
#df_final.to_csv("../data/processed/nmb_observed.csv")